In [1]:

import os, sys
#print('forcing load of development neuron unit')
#sys.path[0]='/home/jovyan/work/scidash'
#sys.path[-1]='/home/jovyan/work/scidash'
#print(sys.path)
#import neuronunit
#print(neuronunit.__file__)
#import pdb
#pdb.set_trace()


def use_dev_packages(dev_packages):
    """
    Adapted from Rickpy https://github.com/rgerkin/rickpy 
    Prepends items in dev_packages to sys.path, and 
    assumes these paths exist in the 
    the user's HOME/mnt/sciunitopt/ directory. 
    Format for dev_packages items is repo/package.
    
    /home/jovyan
    """

    HOME = os.path.expanduser('~')
    
    sp = os.path.join('/','opt/conda/lib/python3.5/site-packages')
    if os.path.exists(sp) and sp not in sys.path:
        sys.path.append(sp)  
    for i,package in enumerate(dev_packages):
        if package.split('/')[-1] not in sys.path[len(dev_packages)-i]:
            sys.path.insert(1,os.path.join(HOME,'mnt/sciunitopt/',package))  

use_dev_packages(['/neuronunit'])
path=os.getcwd()
print(sys.path)

'RS_pop[0]/v'


['', '/neuronunit', '/opt/conda/lib/python35.zip', '/opt/conda/lib/python3.5', '/opt/conda/lib/python3.5/plat-linux', '/opt/conda/lib/python3.5/lib-dynload', '/opt/conda/lib/python3.5/site-packages/cycler-0.10.0-py3.5.egg', '/opt/conda/lib/python3.5/site-packages/setuptools-23.0.0-py3.5.egg', '/opt/conda/lib/python3.5/site-packages', '/opt/conda/lib/python3.5/site-packages/IPython/extensions', '/home/jovyan/.ipython']


'RS_pop[0]/v'

In [2]:

# In[2]:
#get_ipython().magic('matplotlib notebook')
import numpy as np
import matplotlib.pyplot as plt
import quantities as pq
import sciunit
import neuronunit
#from neuronunit import aibs
import pickle as pickle
import pdb

from neuronunit.models.reduced import ReducedModel


# In[3]:

# Replace this with your model path.  
# This example is from https://github.com/OpenSourceBrain/IzhikevichModel.
HOME = os.path.expanduser('~')
LEMS_MODEL_PATH = os.path.join(os.getcwd(),'LEMS_2007One.xml')
model = ReducedModel(LEMS_MODEL_PATH,name='vanilla')



#print(LEMS_MODEL_PATH)


# In[21]:

#vm=np.zeros(13)
from neuronunit.capabilities import spike_functions
#waveforms = spike_functions.get_spike_waveforms(vm)
#np.max(waveforms.data,axis=1)


# In[23]:

#np.max(np.array(waveforms),axis=1)


# In[26]:

#np.max(waveforms,axis=1)


# In[4]:

import quantities as pq
from neuronunit import tests as nu_tests, neuroelectro
neuron = {'nlex_id': 'nifext_50'} # Layer V pyramidal cell

tests = []

dataset_id = 354190013  # Internal ID that AIBS uses for a particular Scnn1a-Tg2-Cre 
                        # Primary visual area, layer 5 neuron.
#observation = aibs.get_observation(dataset_id,'rheobase')
from allensdk.api.queries.cell_types_api import CellTypesApi
from allensdk.ephys.extract_cell_features import get_square_stim_characteristics,\
                                                 get_sweep_from_nwb
from allensdk.core import nwb_data_set

ct = CellTypesApi()

def get_sp(experiment_params,sweep_ids):
    '''
    get sweep parameter
    TODO: move method into neuronunit/aibs.py, as this is a fix for that file.    
    '''
    sweep_num = None
    for sp in experiment_params:
       for i in sweep_ids:
          if sp['id']==i:
              sweep_num = sp['sweep_number']
              found_sp=sp
              break
    if sweep_num is None:
        found_sp=None          
        raise Exception('Sweep with ID %d not found in dataset with ID %d.' % (sweep_id, dataset_id))
    return found_sp


def get_value_dict(experiment_params,sweep_ids,kind=str('rheobase')):
    '''
    return values
    TODO: move method into neuronunit/aibs.py, as this is a fix for that file.
    '''
    if kind == str('rheobase'):
        sp=get_sp(experiment_params,sweep_ids)
        value = sp['stimulus_absolute_amplitude']
        value = np.round(value,2) # Round to nearest hundredth of a pA.
        value *= pq.pA # Apply units.  

        #Need some way to sanitize values in the dictionary below:.
        return {'value': value}              
              


#save some time by pickle loading the content if its available. 
#using allensdk cache would be preferable, but I don't yet understand the syntax.


if os.path.exists(str(os.getcwd())+"/observations.pickle"):
    print('attempting to recover from pickled file')
    with open('observations.pickle', 'rb') as handle:
        observation = pickle.load(handle)

else:
    print('checked path:')
    print(str(os.getcwd())+"/observation.pickle")
    print('no pickled file down loading time intensive')
    experiment_params = ct.get_ephys_sweeps(dataset_id)
    cmd = ct.get_ephys_features(dataset_id)
    sweep_ids=cmd['rheobase_sweep_id'] #Retrieva all of the sweeps corresponding to finding rheobase.
    observation=get_value_dict(experiment_params,sweep_ids)
    with open('observations.pickle', 'wb') as handle:
        pickle.dump(observation, handle)


#Compare differences between Allen Brain derived observations, Neuroelectro derived recordings and 
#Izkevitch model



tests += [nu_tests.RheobaseTest(observation=observation)]
#Edited out below:   



attempting to recover from pickled file


In [3]:
#                     (nu_tests.RestingPotentialTest,None),                         
test_class_params = [(nu_tests.TimeConstantTest,None) ]
                     #(nu_tests.InputResistanceTest,None),
                     #nu_tests.CapacitanceTest,None) ]
                     #(nu_tests.InjectedCurrentAPWidthTest,None),
                     #(nu_tests.InjectedCurrentAPAmplitudeTest,None),
                     #(nu_tests.InjectedCurrentAPThresholdTest,None)]

                     
                     #,
                     

print('neuronunit_generated these tests')
for cls,params in test_class_params:
    print('neuronunit_generated these tests')
    observation = cls.neuroelectro_summary_observation(neuron)
    tests += [cls(observation,params=params)]
  
def update_amplitude(test,tests,score):
    rheobase = score.prediction['value']
    #for i in [3,4,5]:
    #    tests[i].params['injected_square_current']['amplitude'] = rheobase*1.01 # Set current injection to just suprathreshold
    
hooks = {tests[0]:{'f':update_amplitude}}


import pdb
print(tests)
print(hooks)
print(dir(sciunit.TestSuite))
#pdb.set_trace()


suite = sciunit.TestSuite("vm_suite",tests,hooks=hooks)


# In[5]:

model = ReducedModel(LEMS_MODEL_PATH,name='vanilla')
print('interesting that the basic model works')
suite.judge(model)


# In[9]:

test = nu_tests.TimeConstantTest


# In[5]:

models = []


i=0
for vr in np.linspace(-75,-50,6):
    model = ReducedModel(LEMS_MODEL_PATH, 
                         name='V_rest=%dmV' % vr, 
                         attrs={'//izhikevich2007Cell':
                                    {'vr':'%d mV' % vr}
                               });
    #model.skip_run = True
 



    models.append(model)

    check_error = suite.judge(model)
  
print('interesting that judging a list of models with different parameters does not work. Where does it fail?')
print('surely its just the nans or someother weird data type causing a value that is not optimizible in a data range')
print('it would be easier to evaluate if each model was optomized inside the loop')
score_matrix = suite.judge(models)
score_matrix.show_mean = True



neuronunit_generated these tests
neuronunit_generated these tests
Getting data values from neuroelectro.org
http://www.neuroelectro.org/api/1/nes/?nlex=nifext_50&e__name=Membrane+Time+Constant
[RheobaseTest, TimeConstantTest]
{RheobaseTest: {'f': <function update_amplitude at 0x7f831a091ae8>}}
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'description', 'from_observations', 'include_models', 'judge', 'name', 'set_verbose', 'skip_models', 'tests']
interesting that the basic model works


pyNeuroML >>> Executing: (java -Xmx400M  -Djava.awt.headless=true -jar  "/opt/conda/lib/python3.5/site-packages/pyneuroml/lib/jNeuroML-0.8.0-jar-with-dependencies.jar"  "/tmp/vanilla.xml"  -neuron -run -nogui) in directory: .
pyNeuroML >>> Command completed. Output: 
pyNeuroML >>>   jNeuroML >>   jNeuroML v0.8.0
pyNeuroML >>>   jNeuroML >>  (INFO) Reading from: /tmp/vanilla.xml
pyNeuroML >>>   jNeuroML >>  (INFO) Creating NeuronWriter for vanilla_nrn.py
pyNeuroML >>>   jNeuroML >>  (INFO) Adding simulation Component(id=sim1 type=Simulation) of network/component: net1 (Type: network)
pyNeuroML >>>   jNeuroML >>  (INFO) Adding population: RS_pop
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/RS.mod exists but is different
pyNeuroML >>>   jNeuroML >>  (INFO) Adding projections/connections...
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS_Iext.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/RS_Iext.mod exis

pyNeuroML >>> Executing: (java -Xmx400M  -Djava.awt.headless=true -jar  "/opt/conda/lib/python3.5/site-packages/pyneuroml/lib/jNeuroML-0.8.0-jar-with-dependencies.jar"  "/tmp/vanilla.xml"  -neuron -run -nogui) in directory: .
pyNeuroML >>> Command completed. Output: 
pyNeuroML >>>   jNeuroML >>   jNeuroML v0.8.0
pyNeuroML >>>   jNeuroML >>  (INFO) Reading from: /tmp/vanilla.xml
pyNeuroML >>>   jNeuroML >>  (INFO) Creating NeuronWriter for vanilla_nrn.py
pyNeuroML >>>   jNeuroML >>  (INFO) Adding simulation Component(id=sim1 type=Simulation) of network/component: net1 (Type: network)
pyNeuroML >>>   jNeuroML >>  (INFO) Adding population: RS_pop
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/RS.mod exists and is identical
pyNeuroML >>>   jNeuroML >>  (INFO) Adding projections/connections...
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS_Iext.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/RS_Iext.mod exis

pyNeuroML >>> Executing: (java -Xmx400M  -Djava.awt.headless=true -jar  "/opt/conda/lib/python3.5/site-packages/pyneuroml/lib/jNeuroML-0.8.0-jar-with-dependencies.jar"  "/tmp/V_rest=-75mV.xml"  -neuron -run -nogui) in directory: .
pyNeuroML >>> Command completed. Output: 
pyNeuroML >>>   jNeuroML >>   jNeuroML v0.8.0
pyNeuroML >>>   jNeuroML >>  (INFO) Reading from: /tmp/V_rest=-75mV.xml
pyNeuroML >>>   jNeuroML >>  (INFO) Creating NeuronWriter for V_rest=-75mV_nrn.py
pyNeuroML >>>   jNeuroML >>  (INFO) Adding simulation Component(id=sim1 type=Simulation) of network/component: net1 (Type: network)
pyNeuroML >>>   jNeuroML >>  (INFO) Adding population: RS_pop
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/RS.mod exists but is different
pyNeuroML >>>   jNeuroML >>  (INFO) Adding projections/connections...
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS_Iext.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/R

pyNeuroML >>> Executing: (java -Xmx400M  -Djava.awt.headless=true -jar  "/opt/conda/lib/python3.5/site-packages/pyneuroml/lib/jNeuroML-0.8.0-jar-with-dependencies.jar"  "/tmp/V_rest=-75mV.xml"  -neuron -run -nogui) in directory: .
pyNeuroML >>> Command completed. Output: 
pyNeuroML >>>   jNeuroML >>   jNeuroML v0.8.0
pyNeuroML >>>   jNeuroML >>  (INFO) Reading from: /tmp/V_rest=-75mV.xml
pyNeuroML >>>   jNeuroML >>  (INFO) Creating NeuronWriter for V_rest=-75mV_nrn.py
pyNeuroML >>>   jNeuroML >>  (INFO) Adding simulation Component(id=sim1 type=Simulation) of network/component: net1 (Type: network)
pyNeuroML >>>   jNeuroML >>  (INFO) Adding population: RS_pop
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/RS.mod exists and is identical
pyNeuroML >>>   jNeuroML >>  (INFO) Adding projections/connections...
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS_Iext.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/R

...

pyNeuroML >>> Executing: (java -Xmx400M  -Djava.awt.headless=true -jar  "/opt/conda/lib/python3.5/site-packages/pyneuroml/lib/jNeuroML-0.8.0-jar-with-dependencies.jar"  "/tmp/V_rest=-60mV.xml"  -neuron -run -nogui) in directory: .
pyNeuroML >>> Command completed. Output: 
pyNeuroML >>>   jNeuroML >>   jNeuroML v0.8.0
pyNeuroML >>>   jNeuroML >>  (INFO) Reading from: /tmp/V_rest=-60mV.xml
pyNeuroML >>>   jNeuroML >>  (INFO) Creating NeuronWriter for V_rest=-60mV_nrn.py
pyNeuroML >>>   jNeuroML >>  (INFO) Adding simulation Component(id=sim1 type=Simulation) of network/component: net1 (Type: network)
pyNeuroML >>>   jNeuroML >>  (INFO) Adding population: RS_pop
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/RS.mod exists and is identical
pyNeuroML >>>   jNeuroML >>  (INFO) Adding projections/connections...
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS_Iext.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/R

pyNeuroML >>> Executing: (java -Xmx400M  -Djava.awt.headless=true -jar  "/opt/conda/lib/python3.5/site-packages/pyneuroml/lib/jNeuroML-0.8.0-jar-with-dependencies.jar"  "/tmp/V_rest=-55mV.xml"  -neuron -run -nogui) in directory: .
pyNeuroML >>> Command completed. Output: 
pyNeuroML >>>   jNeuroML >>   jNeuroML v0.8.0
pyNeuroML >>>   jNeuroML >>  (INFO) Reading from: /tmp/V_rest=-55mV.xml
pyNeuroML >>>   jNeuroML >>  (INFO) Creating NeuronWriter for V_rest=-55mV_nrn.py
pyNeuroML >>>   jNeuroML >>  (INFO) Adding simulation Component(id=sim1 type=Simulation) of network/component: net1 (Type: network)
pyNeuroML >>>   jNeuroML >>  (INFO) Adding population: RS_pop
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/RS.mod exists but is different
pyNeuroML >>>   jNeuroML >>  (INFO) Adding projections/connections...
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS_Iext.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/R

pyNeuroML >>> Executing: (java -Xmx400M  -Djava.awt.headless=true -jar  "/opt/conda/lib/python3.5/site-packages/pyneuroml/lib/jNeuroML-0.8.0-jar-with-dependencies.jar"  "/tmp/V_rest=-55mV.xml"  -neuron -run -nogui) in directory: .
pyNeuroML >>> Command completed. Output: 
pyNeuroML >>>   jNeuroML >>   jNeuroML v0.8.0
pyNeuroML >>>   jNeuroML >>  (INFO) Reading from: /tmp/V_rest=-55mV.xml
pyNeuroML >>>   jNeuroML >>  (INFO) Creating NeuronWriter for V_rest=-55mV_nrn.py
pyNeuroML >>>   jNeuroML >>  (INFO) Adding simulation Component(id=sim1 type=Simulation) of network/component: net1 (Type: network)
pyNeuroML >>>   jNeuroML >>  (INFO) Adding population: RS_pop
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/RS.mod exists and is identical
pyNeuroML >>>   jNeuroML >>  (INFO) Adding projections/connections...
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS_Iext.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/R

pyNeuroML >>> Executing: (java -Xmx400M  -Djava.awt.headless=true -jar  "/opt/conda/lib/python3.5/site-packages/pyneuroml/lib/jNeuroML-0.8.0-jar-with-dependencies.jar"  "/tmp/V_rest=-50mV.xml"  -neuron -run -nogui) in directory: .
pyNeuroML >>> Command completed. Output: 
pyNeuroML >>>   jNeuroML >>   jNeuroML v0.8.0
pyNeuroML >>>   jNeuroML >>  (INFO) Reading from: /tmp/V_rest=-50mV.xml
pyNeuroML >>>   jNeuroML >>  (INFO) Creating NeuronWriter for V_rest=-50mV_nrn.py
pyNeuroML >>>   jNeuroML >>  (INFO) Adding simulation Component(id=sim1 type=Simulation) of network/component: net1 (Type: network)
pyNeuroML >>>   jNeuroML >>  (INFO) Adding population: RS_pop
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/RS.mod exists but is different
pyNeuroML >>>   jNeuroML >>  (INFO) Adding projections/connections...
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS_Iext.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/R

pyNeuroML >>> Executing: (java -Xmx400M  -Djava.awt.headless=true -jar  "/opt/conda/lib/python3.5/site-packages/pyneuroml/lib/jNeuroML-0.8.0-jar-with-dependencies.jar"  "/tmp/V_rest=-50mV.xml"  -neuron -run -nogui) in directory: .
pyNeuroML >>> Command completed. Output: 
pyNeuroML >>>   jNeuroML >>   jNeuroML v0.8.0
pyNeuroML >>>   jNeuroML >>  (INFO) Reading from: /tmp/V_rest=-50mV.xml
pyNeuroML >>>   jNeuroML >>  (INFO) Creating NeuronWriter for V_rest=-50mV_nrn.py
pyNeuroML >>>   jNeuroML >>  (INFO) Adding simulation Component(id=sim1 type=Simulation) of network/component: net1 (Type: network)
pyNeuroML >>>   jNeuroML >>  (INFO) Adding population: RS_pop
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/RS.mod exists and is identical
pyNeuroML >>>   jNeuroML >>  (INFO) Adding projections/connections...
pyNeuroML >>>   jNeuroML >>  (INFO) -- Writing to mod: /tmp/RS_Iext.mod
pyNeuroML >>>   jNeuroML >>  (INFO) File /tmp/R

In [4]:
print(sys.path)



['', '/neuronunit', '/opt/conda/lib/python35.zip', '/opt/conda/lib/python3.5', '/opt/conda/lib/python3.5/plat-linux', '/opt/conda/lib/python3.5/lib-dynload', '/opt/conda/lib/python3.5/site-packages/cycler-0.10.0-py3.5.egg', '/opt/conda/lib/python3.5/site-packages/setuptools-23.0.0-py3.5.egg', '/opt/conda/lib/python3.5/site-packages', '/opt/conda/lib/python3.5/site-packages/IPython/extensions', '/home/jovyan/.ipython']


In [5]:
score_matrix.sortable = True
score_matrix



<IPython.core.display.Javascript object>

,Mean,RheobaseTest,TimeConstantTest
V_rest=-75mV,0.835,Ratio = 1.41,Z = -0.23
V_rest=-70mV,0.950,Ratio = 1.01,Z = -0.12
V_rest=-65mV,0.901,Ratio = 0.68,Z = 0.04
V_rest=-60mV,0.699,Ratio = 0.41,Z = 0.28
V_rest=-55mV,0.413,Ratio = 0.21,Z = 0.68
V_rest=-50mV,0.082,Ratio = 0.08,Z = 1.39


In [6]:
'''
def plot_spiketrains(segment):
    for spiketrain in segment.spiketrains:
        y = np.ones_like(spiketrain) * spiketrain.annotations['source_id']
        plt.plot(spiketrain, y, '.')
        plt.ylabel(segment.name)
        plt.setp(plt.gca().get_xticklabels(), visible=False)


def plot_signal(signal, index, colour='b'):
    label = "Neuron %d" % signal.annotations['source_ids'][index]
    plt.plot(signal.times, signal[:, index], colour, label=label)
    plt.ylabel("%s (%s)" % (signal.name, signal.units._dimensionality.string))
    plt.setp(plt.gca().get_xticklabels(), visible=False)
    plt.legend()

n_panels = sum(a.shape[1] for a in data_out.segments[0].analogsignals) + 2
plt.subplot(n_panels, 1, 1)
plot_spiketrains(spikes_in.segments[0])
plt.subplot(n_panels, 1, 2)
plot_spiketrains(data_out.segments[0])
panel = 3
for array in data_out.segments[0].analogsignals:
    for i in range(array.shape[1]):
        plt.subplot(n_panels, 1, panel)
        plot_signal(array, i, colour='bg'[panel % 2])
        panel += 1
plt.xlabel("time (%s)" % array.times.units._dimensionality.string)
plt.setp(plt.gca().get_xticklabels(), visible=True)

plt.show()
'''

'\ndef plot_spiketrains(segment):\n    for spiketrain in segment.spiketrains:\n        y = np.ones_like(spiketrain) * spiketrain.annotations[\'source_id\']\n        plt.plot(spiketrain, y, \'.\')\n        plt.ylabel(segment.name)\n        plt.setp(plt.gca().get_xticklabels(), visible=False)\n\n\ndef plot_signal(signal, index, colour=\'b\'):\n    label = "Neuron %d" % signal.annotations[\'source_ids\'][index]\n    plt.plot(signal.times, signal[:, index], colour, label=label)\n    plt.ylabel("%s (%s)" % (signal.name, signal.units._dimensionality.string))\n    plt.setp(plt.gca().get_xticklabels(), visible=False)\n    plt.legend()\n\nn_panels = sum(a.shape[1] for a in data_out.segments[0].analogsignals) + 2\nplt.subplot(n_panels, 1, 1)\nplot_spiketrains(spikes_in.segments[0])\nplt.subplot(n_panels, 1, 2)\nplot_spiketrains(data_out.segments[0])\npanel = 3\nfor array in data_out.segments[0].analogsignals:\n    for i in range(array.shape[1]):\n        plt.subplot(n_panels, 1, panel)\n        